<a href="https://colab.research.google.com/github/HJKQWE/PDF_table_list_to_exel/blob/main/PDF_table_list_to_exel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
#!pip install paddlepaddle-gpu==2.3.2 cudatoolkit==10.2

In [55]:
!pip install paddlepaddle-gpu==2.3.0.post110 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# pdf2image

## Installation

In [56]:
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

Hit:1 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.86.1-0ubuntu1.1).
0 upgraded, 0 newly installed, 0 to re

## Conversion

In [57]:
from pdf2image import convert_from_path

In [ ]:
images = convert_from_path('/content/drive/MyDrive/ГеоКин/Белозерско-Чубовское (501)-2018.pdf')

In [ ]:
!mkdir pages

In [ ]:
for i in range(len(images)):
  images[i].save('pages/page'+str(i)+'.jpg', 'JPEG')

# Layout

## Installation

In [10]:
#!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.6"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.9/445.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery 3.10.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google-cloud-bigquery-connection 1.12.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.
google

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 46611, done.
remote: Counting objects: 100% (462/462), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 46611 (delta 255), reused 315 (delta 178), pack-reused 46149
Receiving objects: 100% (46611/46611), 342.92 MiB | 18.02 MiB/s, done.
Resolving deltas: 100% (32692/32692), done.
Updating files: 100% (1621/1621), done.


In [11]:
a=5

In [12]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

--2023-07-05 11:30:24--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl’

layoutparser-0.0.0- 100%[===================>]  18.26M  4.74MB/s    in 19s     

2023-07-05 11:30:45 (1007 KB/s) - ‘layoutparser-0.0.0-py3-none-any.whl’ saved [19145360/19145360]

Processing ./layoutparser-0.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31531 sha256=0a0070051c7b330ef5981a3ceae0fb589cab8f810e914ffb6941976348aac908
  Stored in directory: /root/.cache/pip/wheel

## Table Extraction

In [13]:
import cv2
import layoutparser as lp
image = cv2.imread("/content/pages/page0.jpg")

image = image[..., ::-1]

# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.5,
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library
# detect
layout = model.detect(image)

download https://paddle-model-ecology.bj.bcebos.com/model/layout-parser/ppyolov2_r50vd_dcn_365e_publaynet.tar to /root/.paddledet/inference_model/ppyolov2_r50vd_dcn_365e_publaynet/ppyolov2_r50vd_dcn_365e_publaynet_infer/ppyolov2_r50vd_dcn_365e_publaynet.tar


100%|██████████| 221M/221M [00:36<00:00, 6.09MiB/s]


In [14]:
layout

Layout(_blocks=[TextBlock(block=Rectangle(x_1=232.3773193359375, y_1=226.898681640625, x_2=1481.6829833984375, y_2=2119.2275390625), text=None, id=None, type=Table, parent=None, next=None, score=0.978708803653717), TextBlock(block=Rectangle(x_1=560.4517822265625, y_1=198.8157196044922, x_2=1078.5328369140625, y_2=223.45167541503906), text=None, id=None, type=Text, parent=None, next=None, score=0.9131478667259216)], page_data={})

In [15]:
x_1=0
y_1=0
x_2=0
y_2=0

for l in layout:
  #print(l)
  if l.type == 'Table':
    x_1 = int(l.block.x_1)
    print(l.block.x_1)
    y_1 = int(l.block.y_1)
    x_2 = int(l.block.x_2)
    y_2 = int(l.block.y_2)

    break

232.37732


In [16]:
print(x_1,y_1,x_2,y_2)

232 226 1481 2119


In [17]:
im = cv2.imread('/content/pages/page0.jpg')

In [18]:
cv2.imwrite('ext_im.jpg', im[y_1:y_2,x_1:x_2])

True

# Text Detection and Recognition

In [19]:
from paddleocr import PaddleOCR, draw_ocr

In [20]:
ocr = PaddleOCR(lang='ru')
image_path = '/content/ext_im.jpg'
image_cv = cv2.imread(image_path)
image_height = image_cv.shape[0]
image_width = image_cv.shape[1]
output = ocr.ocr(image_path)[0]

download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/Multilingual_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer/Multilingual_PP-OCRv3_det_infer.tar


100%|██████████| 3.85M/3.85M [00:13<00:00, 285kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/multilingual/cyrillic_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/cyrillic/cyrillic_PP-OCRv3_rec_infer/cyrillic_PP-OCRv3_rec_infer.tar


100%|██████████| 9.98M/9.98M [00:11<00:00, 895kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:13<00:00, 158kiB/s]

[2023/07/05 11:32:30] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/ml/Multilingual_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/cyrillic/cyrillic_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_

[2023/07/05 11:32:31] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/07/05 11:32:31] ppocr DEBUG: dt_boxes num : 141, elapse : 0.21884942054748535
[2023/07/05 11:32:32] ppocr DEBUG: rec_res num  : 141, elapse : 0.48812103271484375


In [21]:
print(output)

[[[[6.0, 30.0], [59.0, 30.0], [59.0, 51.0], [6.0, 51.0]], ('No I', 0.8794370293617249)], [[[390.0, 35.0], [482.0, 35.0], [482.0, 49.0], [390.0, 49.0]], ('оказађели', 0.9429894685745239)], [[[837.0, 35.0], [866.0, 35.0], [866.0, 51.0], [837.0, 51.0]], ('Ed.', 0.9241485595703125)], [[[868.0, 34.0], [956.0, 34.0], [956.0, 53.0], [868.0, 53.0]], ('измерения', 0.9984573721885681)], [[[997.0, 35.0], [1071.0, 35.0], [1071.0, 49.0], [997.0, 49.0]], ('начения', 0.9616872668266296)], [[[1075.0, 34.0], [1226.0, 34.0], [1226.0, 53.0], [1075.0, 53.0]], ('нароектныйгод', 0.9802488088607788)], [[[129.0, 160.0], [176.0, 160.0], [176.0, 174.0], [129.0, 174.0]], ('Hефт', 0.9404447078704834)], [[[870.0, 152.0], [923.0, 152.0], [923.0, 172.0], [870.0, 172.0]], ('Pyo.т', 0.863427996635437)], [[[211.0, 162.0], [299.0, 162.0], [299.0, 175.0], [211.0, 175.0]], ('знтренне', 0.9633377194404602)], [[[1097.0, 156.0], [1151.0, 156.0], [1151.0, 175.0], [1097.0, 175.0]], ('зђ', 0.911433756351471)], [[[129.0, 191.0],

In [22]:
boxes = [line[0] for line in output]
texts = [line[1][0] for line in output]
probabilities = [line[1][1] for line in output]

In [23]:
image_boxes = image_cv.copy()


In [24]:
for box,text in zip(boxes,texts):
  cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
  cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

In [25]:
cv2.imwrite('detections.jpg', image_boxes)

True

# Reconstruction

## Get Horizontal and Vertical Lines

In [26]:
im = image_cv.copy()

In [27]:
horiz_boxes = []
vert_boxes = []

for box in boxes:
  x_h, x_v = 0,int(box[0][0])
  y_h, y_v = int(box[0][1]),0
  width_h,width_v = image_width, int(box[2][0]-box[0][0])
  height_h,height_v = int(box[2][1]-box[0][1]),image_height

  horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
  vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

  cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
  cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)


In [28]:
cv2.imwrite('horiz_vert.jpg',im)

True

## Non-Max Suppression

In [29]:
horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [30]:
horiz_lines = np.sort(np.array(horiz_out))
print(horiz_lines)

[  3   8  11  15  16  18  23  26  30  32  34  36  37  38  40  41  43  45
  47  52  53  57  60  65  66  68  71  73  76  81  83  87  90  92  93  94
  96  99 103 104 110 112 118 120 123 130 135]


In [31]:
im_nms = image_cv.copy()

In [32]:
for val in horiz_lines:
  cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)


In [33]:
cv2.imwrite('im_nms.jpg',im_nms)

True

In [34]:
vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
)

In [35]:
print(vert_out)

tf.Tensor([130   3  82  23   5  21  60 104   4  52   2   0  61  79  51], shape=(15,), dtype=int32)


In [36]:
vert_lines = np.sort(np.array(vert_out))
print(vert_lines)

[  0   2   3   4   5  21  23  51  52  60  61  79  82 104 130]


In [37]:
for val in vert_lines:
  cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)


In [38]:
cv2.imwrite('im_nms.jpg',im_nms)

True

## Convert to CSV

In [39]:


out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
print(np.array(out_array).shape)
print(out_array)

(47, 15)
[['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [40]:

unordered_boxes = []

for i in vert_lines:
  print(vert_boxes[i])
  unordered_boxes.append(vert_boxes[i][0])

[6, 0, 59, 1893]
[837, 0, 866, 1893]
[868, 0, 956, 1893]
[997, 0, 1071, 1893]
[1075, 0, 1226, 1893]
[86, 0, 125, 1893]
[384, 0, 436, 1893]
[603, 0, 689, 1893]
[687, 0, 765, 1893]
[451, 0, 484, 1893]
[484, 0, 595, 1893]
[250, 0, 304, 1893]
[193, 0, 260, 1893]
[152, 0, 193, 1893]
[329, 0, 387, 1893]


In [41]:
ordered_boxes = np.argsort(unordered_boxes)
print(ordered_boxes)

[ 0  5 13 12 11 14  6  9 10  7  8  1  2  3  4]


In [42]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [43]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [44]:
for i in range(len(horiz_lines)):
  for j in range(len(vert_lines)):
    resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

    for b in range(len(boxes)):
      the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
      if(iou(resultant,the_box)>0.1):
        out_array[i][j] = texts[b]

In [45]:
out_array=np.array(out_array)

In [46]:
out_array

array([['No I', '', '', '', '', '', 'оказађели', 'оказађели', '', '', '',
        'Ed.', 'измерения', 'начения', 'нароектныйгод'],
       ['', '', 'Hефт', 'знтренне', 'знтренне', '', '', '', '', '', '',
        '', 'Pyo.т', '', 'зђ'],
       ['', '', 'Hефты', 'BHеUHем', 'BHеUHем', '', '', '', '', '', '',
        'доллбаpp.', 'доллбаpp.', '', '5O,'],
       ['', '', '', '', 'Bнтреннемрынке', 'Bнтреннемрынке', '', '', '',
        '', '', '', 'Pyo.т', '', ''],
       ['', '', 'природныйгаз', 'природныйгаз', '', '', '', '', '', '',
        '', 'Pyб.J1OоOм', 'Pyб.J1OоOм', '', ''],
       ['', '', 'оренныйгаз', 'оренныйгаз', 'оренныйгаз', '', '', '', '',
        '', '', '', 'Pу6.JOOOM', '', '1736,'],
       ['', 'оля', '', 'HефTI Hа BHешHеM', 'HефTI Hа BHешHеM',
        'HефTI Hа BHешHеM', 'ынке', '', '', '', '', '', '', '', '5D,'],
       ['', '', '', 'обьчу', 'нефтиK', 'дI', '', '', '', '', '', '',
        'руб.т', '', '8'],
       ['', '', '', 'ођђч', 'ођђч', '', '', '', '', '', '', '', '

In [47]:
pd.DataFrame(out_array).to_csv('sample.csv')

## Merging Cells

In [47]:
current_bank=['']*len(out_array[0,:])
print(current_bank)

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [48]:
def empty(arr):
  for i in arr:
    if i=='':
      return True
  return False

In [49]:
cleaned_array=[]

In [50]:
for i in range(len(out_array)):
  if not empty(out_array[i]):
    current_bank=[out_array[i][j] for j in range(len(out_array[i]))]
    cleaned_array.append(current_bank)
    not_empty=True
  else:
    for j in range(len(out_array[i])):
      current_bank[j]+=' '+out_array[i][j]
    print('-->',current_bank)
cleaned_array=np.array(cleaned_array)
print(cleaned_array)

--> [' No I', ' ', ' ', ' ', ' ', ' ', ' оказађели', ' оказађели', ' ', ' ', ' ', ' Ed.', ' измерения', ' начения', ' нароектныйгод']
--> [' No I ', '  ', '  Hефт', '  знтренне', '  знтренне', '  ', ' оказађели ', ' оказађели ', '  ', '  ', '  ', ' Ed. ', ' измерения Pyo.т', ' начения ', ' нароектныйгод зђ']
--> [' No I  ', '   ', '  Hефт Hефты', '  знтренне BHеUHем', '  знтренне BHеUHем', '   ', ' оказађели  ', ' оказађели  ', '   ', '   ', '   ', ' Ed.  доллбаpp.', ' измерения Pyo.т доллбаpp.', ' начения  ', ' нароектныйгод зђ 5O,']
--> [' No I   ', '    ', '  Hефт Hефты ', '  знтренне BHеUHем ', '  знтренне BHеUHем Bнтреннемрынке', '    Bнтреннемрынке', ' оказађели   ', ' оказађели   ', '    ', '    ', '    ', ' Ed.  доллбаpp. ', ' измерения Pyo.т доллбаpp. Pyo.т', ' начения   ', ' нароектныйгод зђ 5O, ']
--> [' No I    ', '     ', '  Hефт Hефты  природныйгаз', '  знтренне BHеUHем  природныйгаз', '  знтренне BHеUHем Bнтреннемрынке ', '    Bнтреннемрынке ', ' оказађели    ', ' оказађ

In [52]:
pd.DataFrame(cleaned_array).to_csv('cleaned.csv')

# Convert to OWL Format

# **CSV to Text**

In [ ]:
import  jpype
import  asposecells


jpype.startJVM()
from asposecells.api import Workbook

workbook = Workbook("/content/sample.csv")
workbook.save("Output.docx")
jpype.shutdownJVM()

ModuleNotFoundError: ignored

# Brighten Image (For Anyone dealing with PDFs created from scanned images)

In [ ]:
from PIL import Image, ImageEnhance

#read the image
im = Image.open("ext_im.jpg")

#image brightness enhancer
enhancer = ImageEnhance.Brightness(im)

factor = 1 #gives original image
im_output = enhancer.enhance(factor)
im_output.save('ext_im-1.jpg')

factor = 1.5## brightens the image
im_output = enhancer.enhance(factor)
im_output.save('ext_im-2.jpg')


In [1]:
!pip install easyocr

In [2]:
import easyocr

In [4]:
def text_recognition(file_path):
    reader = easyocr.Reader(['ru'])
    result = reader.readtext(file_path, detail = 0, paragraph=True)

    return result

def main():
    file_path = input('Insert path:')
    print(text_recognition(file_path=file_path))

if __name__== "__main__":
    main()


Insert path:/content/2023-07-05_15-32-21.png
['Таблица 2.1', 'Состояние запасов растворенного газа Славкинского месторождения', 'Начальые нзнлскасчые ктшпаска мли Утверждениые Тскущне нлкЛскясмыс Накопленная Роснедра " На государстненном злпасы (без учста добыча на (протокол Ха 03-18/493- балансе на 01.01.2018 лобычи 2018) млн; м\' 01,01.2018, пр от 07.092018 г ) МЛН:М А+Н; А+В1 А+В', 'Продуктивные отложення (пласты) объскти, мссторожленнс целом', "Лицснзноннын участок (Липситня €МР 02120 Ю22 Золотарснскос поднятис ПАСТ А-7 Московскин ярус С2ш ПЛАСТ А4 Башкирскии ярус С2Ь ПЛАСТ Б-0 Визсиский ~рус Су ПЛАСТ Б-2 Визснский ~Р€ (у ПЛАСТ В-1 Турнсйский ~.; С1 Итого по Золоваревскому подичтию Сланкинскос 1о [ЛАСТ А-3 Московскни ~рус С2ш ПАСТ А-4 Башкирский ярус €2Ь 6 ПЛАСТ Б-2 Витсйский ярус Су ПЛАСТ Б-3 Витсйский ярус €^у ' Итого ло Славкннскомуполиятню ' Вссто мссторожлсниюь Лнцсшия СИРМ 02120 нэ Нерасираеиныи фомд запасы булут постанлены на госбаланс 01,01,2019 гоца"]
